In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

import numpy as np
import tensorflow as tf
import os.path as osp
from collections import defaultdict

from general_tools.notebook.tf import reset_tf_graph

Picking GPU 3


In [2]:
import helper

from tf_lab.diff_maps.in_out import raw_data, produce_net_data,\
prep_splits_labels_for_task, produce_diff_maps, classes_of_tasks

from tf_lab.diff_maps.basic_nets import pc_net, diff_mlp_net, pc_versions, diff_conv_net
from tf_lab.diff_maps.basic_nets import Basic_Net

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
def average_l2_distance(gt_vecs, pred_vecs):
    return np.mean(np.sqrt(np.sum(np.square((gt_vecs - pred_vecs)), axis=1)))

def score_net(task, stats):
    train_s = np.array(stats['train'])
    test_s = np.array(stats['test'])
    val_s = np.array(stats['val'])

    if task == 'regression':
        seek = np.min(val_s)
    else:
        seek = np.max(val_s)

    val_best_all = np.where(val_s == seek)[0]
    val_maximizer = val_best_all[0]
    test_best = test_s[val_maximizer]
    train_best = train_s[val_maximizer]
    gen_error = test_best - train_best   
    return len(val_best_all), val_maximizer, gen_error[0], test_best[0]

In [5]:
top_mesh_dir = '/orions4-zfs/projects/optas/DATA/Meshes/SCAPE_8_poses/'
gt_param_f = osp.join(top_mesh_dir, 'gt_shape_params.mat')

In [6]:
total_shapes = helper.total_shapes
n_pose_classes = helper.n_pose_classes

In [22]:
use_pc = True
knn = 20
arch = 'conv'

In [23]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/OUT/latent_diff_maps/experiments/SCAPE_8_poses'
synced_bases_file = osp.join(top_data_dir, '50_extract_%d_knn_50_fmapd.mat' % (knn,) )

if arch == 'mlp':
    n_consistent = [5, 10, 20, 30, 40, 50]
else:
    n_consistent = [20, 40, 50]
    
seeds = [42, 100, 666, 1821, 2004]

In [24]:
sub_member_per_class = 50
n_shapes = sub_member_per_class * n_pose_classes
val_per = 0.10
test_per = 0.15
train_per = 1.0 - (val_per + test_per)
n_pc_points = 1024
task = 'regression'
mean_norm_diffs = True

In [25]:
if use_pc:
    learning_rate = 0.005
    batch_size = 50
    pc_version = 'v2'
else:
    learning_rate = 0.007
    batch_size = 50

In [26]:
gt_latent_params, in_pcs, pose_labels = \
raw_data(top_mesh_dir, gt_param_f, sub_member_per_class, n_pc_points)

n_classes = classes_of_tasks(task)

In [30]:
# n_consistent = [0.001, 0.002, 0.005, 0.007, 0.01]

In [27]:
verbose = False
train_epochs = 1000
stats_over_reps = defaultdict(list)

for seed in seeds:
    print seed
    np.random.seed(seed)
    for n_cons in n_consistent:
        # Prepare splits
        diff_maps = produce_diff_maps(synced_bases_file, n_cons, n_shapes)
        splits, labels = \
        prep_splits_labels_for_task(task, gt_latent_params, pose_labels, train_per, test_per, seed)
        net_data = produce_net_data(in_pcs, splits, labels, diff_maps, use_pc, mean_norm_diffs)
        
        # Prepare graph
        reset_tf_graph()
        tf.set_random_seed(seed)
        if use_pc:
            n_filters, n_neurons = pc_versions(pc_version)
            net_out, feed_pl, label_pl = pc_net(n_pc_points, task, n_filters, n_neurons, verbose)
        else:
            if arch == 'mlp':
                net_out, feed_pl, label_pl = diff_mlp_net(n_cons, task, verbose)
            elif arch == 'conv':
                net_out, feed_pl, label_pl = diff_conv_net(n_cons, task, verbose)
        
        net = Basic_Net(net_out, feed_pl, label_pl)
        net.define_loss(task)
        net.define_opt(learning_rate)
        net.start_session()        
        
        # Train
        stats = net.train(train_epochs, batch_size, net_data, task, verbose=verbose)
        n_opt, val_maximizer, gen_error, test_best = score_net(task, stats)    
        if verbose:
            print n_opt, val_maximizer, gen_error, test_best    
        stats_over_reps[n_cons].append([n_opt, val_maximizer, gen_error, test_best])    

42
100
666
1821
2004


In [28]:
np.set_printoptions(precision=3)
for n_cons in n_consistent:
    print n_cons, np.mean(stats_over_reps[n_cons], axis=0)[1:], np.std(stats_over_reps[n_cons], axis=0)[2:]

50 [  3.204e+02   3.106e-02   6.364e-02] [ 0.014  0.01 ]
